# Bandstructures for 1D, 2D, and 3D nanowires
For more information see: B. Nijholt, A. R. Akhmerov, *Orbital effect of magnetic field on the Majorana phase diagram*, [arXiv:1509.02675](https://arxiv.org/abs/1509.02675) [[pdf](https://arxiv.org/pdf/1509.02675.pdf)], [Phys. Rev. B 93, 235434 (2016)](http://journals.aps.org/prb/abstract/10.1103/PhysRevB.93.235434).

In [ ]:
from functools import partial
import holoviews as hv
import kwant
import numpy as np
from wires import make_1d_wire, make_2d_wire, make_3d_wire, make_3d_wire_external_sc
import sympy.physics
from scipy.constants import hbar, m_e, eV, physical_constants
import types

class SimpleNamespace(types.SimpleNamespace):
    def update(self, **kwargs):
        self.__dict__.update(kwargs)
        return self

hv.notebook_extension()
%output size=150
%opts Path [aspect='square']
    
# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = SimpleNamespace(
    m=0.015 * m_e,  # effective mass in kg
    a=10,  # lattice spacing in nm
    g=50,  # Lande factor
    hbar=hbar,
    m_e=m_e,
    meV=eV * 1e-3)

constants.t = (hbar ** 2 / (2 * constants.m)) * (1e18 / constants.meV)  # meV * nm^2
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV


def make_params(alpha=20,
                B_x=0,
                B_y=0,
                B_z=0,
                Delta=0.25,
                mu=0,
                orbital=True,
                t=constants.t,
                g=50,
                mu_B=constants.mu_B,
                **kwargs):
    """Function that creates a namespace with parameters.

    Parameters:
    -----------
    alpha : float
        Spin-orbit coupling strength in units of meV*nm.
    B_x, B_y, B_z : float
        The magnetic field strength in the x, y and z direction in units of Tesla.
    Delta : float
        The superconducting gap in units of meV.
    mu : float
        The chemical potential in units of meV.
    orbital : bool
        Switches the orbital effects on and off.
    t : float
        Hopping parameter in meV * nm^2.
    g : float
        Lande g factor.
    mu_B : float
        Bohr magneton in meV/K.

    Returns:
    --------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    """
    p = SimpleNamespace(alpha=alpha,
                              B_x=B_x,
                              B_y=B_y,
                              B_z=B_z,
                              Delta=Delta,
                              mu=mu,
                              orbital=orbital,
                              t=t,
                              g=g,
                              mu_B=mu_B,
                              **kwargs)
    return p


def plot_bands(lead, p, c=constants):
    """
    Plots the bandstructure for lead.

    Parameters:
    -----------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    lead : kwant.builder.InfiniteSystem object
        The finalized infinite system.
    c : SimpleNamespace object
        A namespace container with all constant (fundamental) parameters used.
    """
    Q = np.sqrt(p.mu ** 2 + p.Delta ** 2) - \
        np.sqrt(p.B_x ** 2 + p.B_y ** 2 + p.B_z ** 2) * c.g * c.mu_B / 2
    if Q > 0:
        print("System is trivial (B < sqrt(mu**2 + delta**2))")
    else:
        print("System is topological (B > sqrt(mu**2 + delta**2))")

    bands = kwant.physics.Bands(lead, args=[p])
    momenta = np.linspace(-2, 2, 51)
    evs = np.array([bands(k=k) for k in momenta])
    
    dims = [hv.Dimension(name=r'$k$', unit=r'nm$^{-1}$'),
            hv.Dimension(name=r'$E$', unit=r'meV')]
    
    return hv.Path((momenta, evs), kdims=dims, label="Band structure")

kdims = [hv.Dimension("ylim", range=(0.01, 50)), 
         hv.Dimension(r"$\alpha$", range=(0, 50), unit='meV·nm'), 
         hv.Dimension(r"$\mu$", range=(0.0, 30), unit='meV'), 
         hv.Dimension(r"$B_x$", range=(0, 10), unit='T'), 
         hv.Dimension(r"$B_y$", range=(0, 10), unit='T'), 
         hv.Dimension(r"$B_z$", range=(0, 0.5), unit='T'), 
         hv.Dimension(r"$\Delta$", range=(0, 1), unit='meV'),
         hv.Dimension("orbital", values=[True, False])]


def dm(lead, p, ylim, alpha, mu, B_x, B_y, B_z, Delta, orbital):
    """Simple wrapper function."""
    return plot_bands(lead, p.update(mu=mu, 
                                     alpha=alpha, 
                                     B_x=B_x, 
                                     B_y=B_y, 
                                     B_z=B_z, 
                                     Delta=Delta,
                                     orbital=orbital)).select(E=(-ylim, ylim))

# 1D semiconductor heterostructure with proximity induced superconductivity
$$H(x) = \left[ \frac{p_x^2}{2m} - \mu \right]\sigma_0\otimes\tau_z + \alpha p_x \sigma_y \otimes \tau_z + \frac{1}{2}B_x\sigma_x \otimes \tau_0  + \Delta\sigma_0\otimes\tau_x$$


In [ ]:
lead = make_1d_wire(a=10, L=None)
p = make_params()
hv.DynamicMap(partial(dm, lead, p, orbital=None), kdims=kdims[:-1])

# 2D semiconductor heterostructure with proximity induced superconductivity

In [ ]:
%%opts Path [aspect='square']
lead = make_2d_wire(a=10, L=None)
p = make_params()
hv.DynamicMap(partial(dm, lead, p), kdims=kdims)

# 3D semiconductor heterostructure with proximity induced superconductivity


$$H=\left(\frac{\mathbf{p}^{2}}{2m}-\mu\right)\sigma_{0}\otimes\tau_{z}+\alpha\left(p_{y}\sigma_{x}\otimes\tau_{z}-p_{x}\sigma_{y}\otimes\tau_{z}\right)+\frac{1}{2}B_{x}\sigma_{x}\otimes\tau_{0}+\frac{1}{2}B_{y}\sigma_{y}\otimes\tau_{0}+\frac{1}{2}B_{z}\sigma_{z}\otimes\tau_{0}+\Delta \sigma_{0}\otimes\tau_{x}$$

## 3D wire with constant SC gap, so no external SC

In [ ]:
lead = make_3d_wire(a=10, L=None)
p = make_params(V=lambda x, y, z: 0)
hv.DynamicMap(partial(dm, lead, p), kdims=kdims)

## 3D wire with external SC 

In [ ]:
lead = make_3d_wire_external_sc(a=10)
p = make_params(V=lambda x, y, z: 0, t_interface=7*constants.t/8, A_correction=True)
hv.DynamicMap(partial(dm, lead, p), kdims=kdims)

# Wavefunctions
Plotting the wavefunction in the cross section of a 3D infinite lead.

In [ ]:
def wavefunctions(lead, momentum, p):
    h, t = lead.cell_hamiltonian(args=[p]), lead.inter_cell_hopping(args=[p])
    h_k = lambda k: h + t * np.exp(1j * k) + t.T.conj() * np.exp(-1j * k)
    vals, vecs = np.linalg.eigh(h_k(momentum))
    indxs = np.argsort(abs(vals))
    vecs = vecs[:, indxs]
    vals = vals[indxs]
    return vals, vecs


def densities(lead, momentum, p):
    coord = np.array([i.pos for i in lead.sites])
    yz = coord[coord[:, 0] == 0][:, 1:]
    vals, vecs = wavefunctions(lead, momentum, p)
    num_orbital = 4
    densities = np.linalg.norm(vecs.reshape(-1, num_orbital, len(vecs)), axis=1)**2
    return yz, vals, densities.T


num_wfs = 40
yz, energies, densities = densities(lead, 0, p)
wfs = [kwant.plotter.mask_interpolate(yz, density, oversampling=1)[0] for density in densities[:num_wfs]]

In [ ]:
%%opts Image {+framewise} (cmap='viridis') [xaxis=None yaxis=None bgcolor='white']

ims = {E: hv.Image(wf) for E, wf in zip(energies, wfs)}
hv.HoloMap(ims, kdims=[hv.Dimension('$E$', unit='meV')])